## Zaawansowane Metody Inteligencji Obliczeniowej
# Zadanie domowe 1
### Prowadzący: Michał Kempka, Marek Wydmuch
### Autor: Adam Gorgoń 145278

## Wprowadzenie

Całe zadanie jest oparte o różne wersje środowiska `VacuumEnvironemnt`, które rozważaliśmy na zajęciach.
Środowisko zaimplementowane jest w bibliotece aima3 (https://github.com/ArtificialIntelligenceToolkit/aima3),
która zawiera kod do książki "Artificial Intelligence: A Modern Approach".

#### Uwaga: Możesz dowolnie modyfikować elementy tego notebooka (wstawiać komórki i zmieniać kod) o ile nie napisano gdzieś inaczej.

In [1]:
# Zainstaluj bibliotekę OpenAI Gym
# !pip install aima3

In [2]:
# Zaimportuj wszystkie jego elementy
import collections

collections.Callable = collections.abc.Callable

from aima3.agents import *

Wszystkie używane przez nas elementy biblioteki są zaimplementowane w pliku: https://github.com/ArtificialIntelligenceToolkit/aima3/blob/master/aima3/agents.py

# Zad. 1 - Cechy środowiska odkurzacza (1 pkt.)

Wypisz cechy poniżej używanego środowiska zgodnie z klasyfikacją z wykładu 1.
Dla ciągłości/dyskretności określ cechy osobno w stosunku do czasu, akcji i przestrzeni stanów.
W razie wątpliwości uzasadnij swój wybór.

Odpowiedź:
* częsciowo obserwowalne
* deterministyczne
* statyczne
* jednoagentowy
* czas - ciągły
* akcja - dyskretny
* przestrzeń stanów - dyskretny

Tip: Możesz sprawdź implementacje środowiska w pliku podanym powyżej, lub wywnioskować cechy na wykonując poniższe fragmenty kodu.

In [3]:
# Stwórz nowe środowisko świata odkurzacza
env = TrivialVacuumEnvironment()

In [4]:
# Sprawdź aktualny status środowiska
env.status

{(0, 0): 'Clean', (1, 0): 'Dirty'}

In [5]:
# Utwórz agenta refleksyjnego
agent = ReflexVacuumAgent()
agent.is_alive()

True

In [6]:
# Dodaj agenta do środowiska. Owijamy go w TraceAgent'a, żeby zobaczyć co robi.
env.add_thing(TraceAgent(agent))

In [7]:
# Zobacz gdzie jest agent
for loc in [loc_A, loc_B]:
    print('loc {0}: {1}'.format(loc, env.list_things_at(loc)))
# Lub:
agent.location

loc (0, 0): []
loc (1, 0): [<Agent>]


(1, 0)

In [8]:
# Wykonaj 10 kroków
env.run(10)

<Agent> perceives ((1, 0), 'Dirty') and does Suck
<Agent> perceives ((1, 0), 'Clean') and does Left
<Agent> perceives ((0, 0), 'Clean') and does Right
<Agent> perceives ((1, 0), 'Clean') and does Left
<Agent> perceives ((0, 0), 'Clean') and does Right
<Agent> perceives ((1, 0), 'Clean') and does Left
<Agent> perceives ((0, 0), 'Clean') and does Right
<Agent> perceives ((1, 0), 'Clean') and does Left
<Agent> perceives ((0, 0), 'Clean') and does Right
<Agent> perceives ((1, 0), 'Clean') and does Left


In [9]:
# Sprawdź jak środowisko oceniło jakość agenta.
agent.performance

1

In [10]:
# Moglibyśmy ocenić oczekiwaną jakość agenta dokładniej..., ale tylko ją oszacujemy (1000 powtórzeń).
# Zakładamy, że symulacja trwa 50 kroków.

compare_agents(TrivialVacuumEnvironment, [ReflexVacuumAgent, ModelBasedVacuumAgent], 1000, 50)

[(<function aima3.agents.ReflexVacuumAgent()>, -38.945),
 (<function aima3.agents.ModelBasedVacuumAgent()>, 9.05)]

# Zad. 2 - Cechy zmodyfikowanego środowisko odkurzacza (1 pkt).

Wypisz cechy poniżej używanego środowiska zgodnie z klasyfikacją z wykładu 1.
Dla ciągłości/dyskretności określ cechy osobno w stosunku do czasu, akcji i przestrzeni stanów.
W razie wątpliwości uzasadnij swój wybór.

Odpowiedź:
* częsciowo obserwowalne
* deterministyczne
* statyczne
* jednoagentowy
* czas - ciągły
* akcja - dyskretny
* przestrzeń stanów - dyskretny

In [11]:
# Rozszerzmy implementacje TrivialVacuumEnvironment

import random


class TrivialVacuumEnvironmentWithCats(TrivialVacuumEnvironment):
    def __init__(self, random_dirt_prob=0.05, seed=None):
        super(TrivialVacuumEnvironmentWithCats, self).__init__()
        self.random = random.Random(seed)
        self.random_dirt_prob = random_dirt_prob

    def execute_action(self, agent, action):
        """Change agent's location and/or location's status; track performance; add dirt;
        Score 10 for each dirt cleaned; -1 for each move."""
        # Same as in case of TrivialVacuumEnvironment
        if action == 'Right':
            agent.location = loc_B
            agent.performance -= 1
        elif action == 'Left':
            agent.location = loc_A
            agent.performance -= 1
        elif action == 'Suck':
            if self.status[agent.location] == 'Dirty':
                agent.performance += 10
            self.status[agent.location] = 'Clean'

        # Cats can make either location dirty
        for loc in [loc_A, loc_B]:
            if self.random.random() < self.random_dirt_prob:
                self.status[loc] = 'Dirty'

In [12]:
# Przetestujmy domyślnych agentów w nowym środowisku

def env_factory():
    return TrivialVacuumEnvironmentWithCats(random_dirt_prob=0.05)


compare_agents(env_factory, [ReflexVacuumAgent, ModelBasedVacuumAgent], 1000, 50)

[(<function aima3.agents.ReflexVacuumAgent()>, 12.645),
 (<function aima3.agents.ModelBasedVacuumAgent()>, 33.59)]

# Zad. 3 - Własny program agenta (8 pkt.)

Napisz program agenta, który będzie (średnio) dużo lepszy dla tego środowiska (50 kroków, z random_dirt_prob=0.05) niż ModelBaseVacuumAgent oraz ReflexVacuumAgent. Opisz działanie swojego programu, na podstawie jaki przesłanek on działa, jakbyś go zmodyfikował gdyby prawdopodobieństwo zabrudzenia pokoju (random_dirt_prob) się zmieniło?

Punktacja za wynik (sprawdzarka zrobi 50000 powtórzeń):
* \> 41: 1 pkt.
* \> 42: 2 pkt.
* \> 43: 3 pkt.
* \> 44: 4 pkt.
* \> 45: 5 pkt.
* \> 46: 6 pkt.

\+ 2 pkt. za opis.

#### Uwaga: nie zmieniaj nazwy klasy `MyVacuumAgent`. Nie dopisuj do komórki z klasą innego kodu. Możesz zdefiniować funkcje pomocnicze w tej samej komórce (sprawdzarka wyciągnie ze zgłoszonego notebooka wyłącznie komórkę z klasę o nazwie `MyVacuumAgent` do sprawdzenia).

In [24]:
# Klasa MyVacuumAgent wypełniona przykładowym kodem agenta z modelem

def MyVacuumAgent():
    step = 0
    i = 10

    def program(percept):
        # Zwróć jedną z dostępnych akcji: {'Suck', 'Right', 'Left', 'NoOP'}
        nonlocal i, step
        step += 1
        location, status = percept
        if status == 'Dirty':
            return 'Suck'
        elif step == 49:
            return 'Right' if location == loc_A else 'Left'
        elif location == loc_A and i == 0:
            i = 10
            return 'Right'
        elif location == loc_B and i == 0:
            i = 10
            return 'Left'
        else:
            i -= 1
            return 'NoOP'

    return Agent(program)

In [25]:
# Przetestuj swojego agenta
def env_factory():
    return TrivialVacuumEnvironmentWithCats(random_dirt_prob=0.05)


compare_agents(env_factory, [MyVacuumAgent], 50000, 50)

[(<function __main__.MyVacuumAgent()>, 45.27788)]

In [15]:
for val in range(50):
    print(val)
    def MyVacuumAgent():
        i = val

        def program(percept):
            # Zwróć jedną z dostępnych akcji: {'Suck', 'Right', 'Left', 'NoOP'}
            nonlocal i
            location, status = percept
            if status == 'Dirty':
                return 'Suck'
            elif location == loc_A and i == 0:
                i = val
                return 'Right'
            elif location == loc_B and i == 0:
                i = val
                return 'Left'
            else:
                i -= 1
                return 'NoOP'

        return Agent(program)

    def env_factory():
        return TrivialVacuumEnvironmentWithCats(random_dirt_prob=0.05)

    print(compare_agents(env_factory, [MyVacuumAgent], 1000, 50))

0
[(<function MyVacuumAgent at 0x7f8fa43a4c10>, 13.052)]
1
[(<function MyVacuumAgent at 0x7f8fa5c67f40>, 32.592)]
2
[(<function MyVacuumAgent at 0x7f8fa4328550>, 39.977)]
3
[(<function MyVacuumAgent at 0x7f8fa43824d0>, 42.821)]
4
[(<function MyVacuumAgent at 0x7f8fa43a4ca0>, 43.937)]
5
[(<function MyVacuumAgent at 0x7f8fa5c67eb0>, 45.061)]
6
[(<function MyVacuumAgent at 0x7f8fa43280d0>, 45.356)]
7
[(<function MyVacuumAgent at 0x7f8fa43827a0>, 45.54)]
8
[(<function MyVacuumAgent at 0x7f8fa43a4c10>, 45.083)]
9
[(<function MyVacuumAgent at 0x7f8fa5c67f40>, 44.041)]
10
[(<function MyVacuumAgent at 0x7f8fa4328550>, 44.072)]
11
[(<function MyVacuumAgent at 0x7f8fa43824d0>, 42.258)]
12
[(<function MyVacuumAgent at 0x7f8fa43a4ca0>, 43.62)]
13
[(<function MyVacuumAgent at 0x7f8fa5c67eb0>, 44.95)]
14
[(<function MyVacuumAgent at 0x7f8fa43280d0>, 43.071)]
15
[(<function MyVacuumAgent at 0x7f8fa43827a0>, 39.866)]
16
[(<function MyVacuumAgent at 0x7f8fa43a4c10>, 40.29)]
17
[(<function MyVacuumAgent